# Create a Model Inference Nuclio Function

- [Initialize Nuclio Emulation, Environment Variables, and Configuration](#netops-infer-init-func)
  - [Install Required Packages](#netops-infer-install-pkgs)
- [Set a cron Function Trigger](#netops-infer-set-func-trigger)
- [Initialize the Function](#netops-infer-func-init)
- [Deploy the Function](#netops-infer-func-deploy)

<a id="netops-infer-init-func"></a>
## Initialize Nuclio Emulation, Environment Variables, and Configuration

> **Note:** Use `# nuclio: ignore` for sections that don't need to be copied to the function.

In [11]:
# nuclio: ignore
import nuclio

In [12]:
# nuclio: ignore
from os import environ
version = '1.0'
environ['MODEL_FILE'] = 'netops.v{}.model'.format(version)

In [13]:
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c MODEL_FILE=${MODEL_FILE}
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/${MODEL_FILE}

%nuclio: setting 'MODEL_FILEPATH' environment variable


<a id="netops-infer-install-pkgs"></a>
### Install Required Packages

`%nuclio cmd` allows you to run image build instructions and install packages.

> **Note:** When using the `-c` option, commands will only run in Nuclio and not locally.

In [14]:
%%nuclio cmd -c
pip install pandas
pip install sklearn
pip install v3io_frames
pip install dask_ml
pip install dask['complete']
apt-get update && apt-get install -y wget
mkdir -p /tmp/mlmodel

In [15]:
# Copy the model file into the function
%nuclio cmd -c wget -O /tmp/mlmodel/${MODEL_FILE} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081/users/${V3IO_USERNAME}/demos/netops/models/${MODEL_FILE}

<a id="netops-infer-set-func-trigger"></a>
## Set a cron Function Trigger

Set a cron function trigger with 1-minute interval:

In [16]:
%%nuclio config 
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "600s"
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '600s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


<a id="netops-infer-func-init"></a>
## Initialize the Function

In [17]:
import pandas as pd
import pickle
import sklearn
from dask_ml.wrappers import ParallelPostFit
import sys
sys.path.append('../../')
import v3io_frames_local as v3f
from os import environ

# Define a V3IO TSDB client
client = v3f.Client('framesd:8081')

# Load the model
with open(environ['MODEL_FILEPATH'], 'rb+') as f:
    model = pickle.load(f) 
    
clf = ParallelPostFit(estimator=model,
                      scoring='accuracy')

In [18]:
%%nuclio handler
# Load last hour data
df = client.read(backend='tsdb', query='select avg(cpu_utilization) , avg(latency) , avg(packet_loss) , avg(throughput) from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='1h')
df['prediction'] = clf.predict(df)
#df.reset_index(inplace=True)
#client.write(backend='tsdb', table='netops_predictions', dfs=[df], index_cols=['company', 'data_center', 'device', 'time'])
str(df['prediction']) # nuclio: return

'time                 company         data_center     device       \n2020-01-10 17:54:43  Roberts_PLC     Gregory_Tunnel  3565281248497    0.0\n2020-01-10 18:54:43  Roberts_PLC     Gregory_Tunnel  3565281248497    0.0\n2020-01-10 17:54:43  Douglas-Holmes  Lopez_Summit    7236123187473    0.0\n2020-01-10 18:54:43  Douglas-Holmes  Lopez_Summit    7236123187473    0.0\n2020-01-10 17:54:43  Roberts_PLC     Gregory_Tunnel  4401357069985    0.0\n2020-01-10 18:54:43  Roberts_PLC     Gregory_Tunnel  4401357069985    0.0\n2020-01-10 17:54:43  Douglas-Holmes  Lopez_Summit    4752014671680    0.0\n2020-01-10 18:54:43  Douglas-Holmes  Lopez_Summit    4752014671680    0.0\n2020-01-10 17:54:43  Roberts_PLC     John_Curve      7026488238274    0.0\n2020-01-10 18:54:43  Roberts_PLC     John_Curve      7026488238274    0.0\n2020-01-10 17:54:43  Douglas-Holmes  Paul_Lane       3232046314748    0.0\n2020-01-10 18:54:43  Douglas-Holmes  Paul_Lane       3232046314748    0.0\n2020-01-10 17:54:43  Roberts_PL

<a id="netops-infer-func-deploy"></a>
## Deploy the Function

In [19]:
%nuclio deploy -n netops-infer -p netops -c

[nuclio] 2020-01-10 18:54:51,130 (info) Build complete
[nuclio] 2020-01-10 18:55:52,772 (error) Failed to deploy. Details:
[18:55:39.657] (E) Caught unhandled exception while initializing [err="No module named 'v3io_frames_local'",
traceback="Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 324, in run_wrapper
    args.trigger_name)
  File "/opt/nuclio/_nuclio_wrapper.py", line 51, in __init__
    self._entrypoint = self._load_entrypoint_from_handler(handler)
  File "/opt/nuclio/_nuclio_wrapper.py", line 231, in _load_entrypoint_from_handler
    module = __import__(module_name)
  File "/opt/nuclio/04-infer.py", line 9, in <module>
    import v3io_frames_local as v3f
ModuleNotFoundError: No module named 'v3io_frames_local'
]



%nuclio: error: cannot deploy 


In [20]:
# nuclio: ignore
client.read(backend='tsdb', query='select avg(cpu_utilization) , avg(latency) , avg(packet_loss) , avg(throughput)  from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='5m').head(10)


,,,,avg(cpu_utilization),avg(latency),avg(packet_loss),avg(throughput)
time,company,data_center,device,,,,
2020-01-10 18:25:52,Douglas-Holmes,Paul_Lane,1483265418393,68.933978,2.313415,0.999977,239.590221
2020-01-10 18:30:52,Douglas-Holmes,Paul_Lane,1483265418393,73.454727,1.413956,0.801538,249.498118
2020-01-10 18:35:52,Douglas-Holmes,Paul_Lane,1483265418393,81.130607,33.228238,11.452649,190.715959
2020-01-10 18:40:52,Douglas-Holmes,Paul_Lane,1483265418393,73.871325,10.117013,5.086630,228.470281
2020-01-10 18:45:52,Douglas-Holmes,Paul_Lane,1483265418393,69.703528,2.132821,0.627302,253.620352
2020-01-10 18:50:52,Douglas-Holmes,Paul_Lane,1483265418393,72.323913,2.087851,0.855528,244.880231
2020-01-10 18:55:52,Douglas-Holmes,Paul_Lane,1483265418393,70.847875,1.775647,1.089348,254.452688
2020-01-10 19:00:52,Douglas-Holmes,Paul_Lane,1483265418393,71.363916,2.127684,0.899277,246.894183
2020-01-10 19:05:52,Douglas-Holmes,Paul_Lane,1483265418393,83.564346,41.571719,17.219803,166.853100
